In [7]:
import requests
from requests.auth import HTTPBasicAuth
import pandas as pd
import unicodedata

In [8]:
# Configuración
wordpress_url = 'https://www.tusitio.com/'
username = 'admin'
password = 'tu pass'
excel_file = 'universidades.xlsx'  # Reemplaza con la ruta a tu archivo de Excel

In [9]:
# Leer datos desde el archivo de Excel
df = pd.read_excel(excel_file)

# Eliminar filas que tienen valores NaN
df = df.dropna(subset=['universidad'])

# Convertir el DataFrame en una lista de diccionarios
universidades = df.to_dict(orient='records')

In [10]:
# Función para normalizar etiquetas (sin acentos y en minúsculas)
def normalizar_etiqueta(etiqueta):
    etiqueta = unicodedata.normalize('NFKD', etiqueta).encode('ASCII', 'ignore').decode('utf-8')
    return etiqueta.lower()

In [11]:
# Función para crear un post
def crear_post(nombre, contenido, etiquetas):
    url = f"{wordpress_url}/wp-json/wp/v2/universidades"
    headers = {
        'Content-Type': 'application/json',
    }
    data = {
        'title': nombre,
        'content': contenido,
        'status': 'publish',
        'tags': etiquetas,
    }

    response = requests.post(url, json=data, headers=headers, auth=HTTPBasicAuth(username, password))
    
    if response.status_code == 201:
        print(f"Post '{nombre}' creado exitosamente.")
        return response.json()
    else:
        print(f"Error al crear el post '{nombre}': {response.content}")
        return None

In [12]:
# Función para obtener el ID de una etiqueta o crearla si no existe
def obtener_o_crear_etiqueta(nombre_etiqueta):
    if not nombre_etiqueta or pd.isna(nombre_etiqueta):
        return None
    
    nombre_etiqueta = normalizar_etiqueta(nombre_etiqueta)

    url = f"{wordpress_url}/wp-json/wp/v2/tags"
    params = {'search': nombre_etiqueta}
    
    response = requests.get(url, params=params, auth=HTTPBasicAuth(username, password))
    
    if response.status_code == 200:
        etiquetas = response.json()
        if etiquetas:
            return etiquetas[0]['id']
        else:
            # Crear la etiqueta si no existe
            data = {'name': nombre_etiqueta}
            response = requests.post(url, json=data, headers={'Content-Type': 'application/json'}, auth=HTTPBasicAuth(username, password))
            
            if response.status_code == 201:
                return response.json()['id']
            else:
                print(f"Error al crear la etiqueta '{nombre_etiqueta}': {response.content}")
                return None
    else:
        print(f"Error al buscar la etiqueta '{nombre_etiqueta}': {response.content}")
        return None


In [13]:
# Crear posts para todas las universidades
for uni in universidades:
    etiquetas = []
    
    # Obtener o crear etiquetas del distrito
    distrito = uni.get('distrito')
    if distrito and not pd.isna(distrito):  # Verificar que el distrito no esté vacío
        etiqueta_id = obtener_o_crear_etiqueta(distrito)
        if etiqueta_id:
            etiquetas.append(etiqueta_id)
    
    # Evaluar las etiquetas 'publica', 'privada', 'online'
    for key in ['publica', 'privada', 'online']:
        if uni.get(key) == 'si':
            etiqueta_id = obtener_o_crear_etiqueta(key)
            if etiqueta_id:
                etiquetas.append(etiqueta_id)
    
    # Crear el post
    nombre = uni['universidad']
    contenido = uni.get('contenido', '')
    crear_post(nombre, contenido, etiquetas)
    
print("Proceso completado")

Post 'UVEG - Universidad Virtual del Estado de Guanajuato' creado exitosamente.
Post 'UDG Virtual ' creado exitosamente.
Post 'UNADM - Universidad SEP en Línea' creado exitosamente.
Post 'SUAyED - UNAM en Línea' creado exitosamente.
Post 'Polivirtual - IPN en Línea' creado exitosamente.
Post 'UAEMex en Línea' creado exitosamente.
Post 'UAEM en Línea' creado exitosamente.
Post 'BUAP en Línea y a Distancia' creado exitosamente.
Post 'UAQ Virtual' creado exitosamente.
Post 'UJED Virtual' creado exitosamente.
Post 'ITSON Virtual ' creado exitosamente.
Post 'UNACH Virtual' creado exitosamente.
Post 'UAEH Virtual ' creado exitosamente.
Post 'UACH Virtual ' creado exitosamente.
Post 'UV en línea' creado exitosamente.
Post 'UJAT en línea ' creado exitosamente.
Post 'UVM en Línea' creado exitosamente.
Post 'UTEL' creado exitosamente.
Post 'Tecmilenio Online' creado exitosamente.
Post 'Universidad Mexicana ' creado exitosamente.
Post 'La Salle en Línea' creado exitosamente.
Post 'Unitec en Línea